# Explore IMDb movies dataset

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
from pandas.io.json import json_normalize
from project_utils import *

%reload_ext lab_black
%load_ext autoreload
%autoreload 2

## Import data

In [ ]:
movies_df = pd.read_csv("./data/tmdb_5000_movies.csv", sep=",", quotechar='"').rename(
    {"id": "movie_id"}, axis=1
)
movies_df.head()

In [ ]:
movies_df.columns

In [ ]:
credits_df = pd.read_csv("./data/tmdb_5000_credits.csv", sep=",", quotechar='"')

In [ ]:
credits_df.columns

## Explore movies data

In [ ]:
movies_df.describe()

### Number of entries with zero values on important features

In [ ]:
print("# of movies with 0 budget: " + str(len(movies_df[movies_df["budget"] == 0])))

In [ ]:
print(
    "# of movies with 0 popularity: "
    + str(len(movies_df[movies_df["popularity"] == 0]))
)

In [ ]:
print("# of movies with 0 revenue: " + str(len(movies_df[movies_df["revenue"] == 0])))

In [ ]:
print("# of movies with 0 runtime: " + str(len(movies_df[movies_df["runtime"] == 0])))

In [ ]:
print(
    "# of movies with 0 vote_average: "
    + str(len(movies_df[movies_df["vote_average"] == 0]))
)

In [ ]:
print(
    "# of movies with 0 vote_count: "
    + str(len(movies_df[movies_df["vote_count"] == 0]))
)

### Numerical features exploration
Plot the histograms of the numerical features to indentify outliers

In [ ]:
np.log10(np.min(movies_df["budget"]) + 1e-6)

In [ ]:
col = movies_df["budget"]
plot_hist(col, "Budget histogram", "budget [$us] (log10 scale)", "count", log=True)

In [ ]:
col = movies_df["popularity"]
plot_hist(col, "Popularity histogram", "popularity (log10 scale)", "count", log=True)

In [ ]:
col = movies_df["revenue"]
plot_hist(col, "Revenue histogram", "revenue (log10 scale)", "count", log=True)

In [ ]:
col = movies_df["vote_average"]
plot_hist(col, "Vote average histogram", "vote_average", "count", log=False, bins=105)

In [ ]:
col = movies_df["vote_count"]
plot_hist(col, "Vote count histogram", "vote_count (log10 scale)", "count", log=True)

### Explore vote count
Since vote_average is not significat when the value vote_count is too low. We need to find a way to normalize this value.
IMDb has already come up with a formula for a normalized rating called "Weighted Rating": <br>

\begin{align*}
    WR = R\frac{v}{v+m}+C\frac{m}{v+m}
\end{align*}

where: <br>
R is the vote_average for the movie <br>
v is the vote_count <br>
m is the minimum vote_count required (parameter that we are going to set manually <br>
C is the mean vote across the whole report <br>

In [ ]:
min_quantile = 0.1
max_quantile = 0.50

To find a good minimum vote_count we are going to check the quantiles 0.1 - 0.5

In [ ]:
vote_count_col = movies_df["vote_count"]
quantiles = np.arange(0.1, max_quantile + 0.01, 0.01)
quantiles_vote_count = vote_count_col.quantile(quantiles)
plt.plot(quantiles, quantiles_vote_count)
plt.title("Quantiles 0.1 to " + str(max_quantile))
plt.xlabel("quantile")
plt.ylabel("value")
plt.grid()
plt.show()

In [ ]:
quantile = 0.212
min_vote_count = vote_count_col.quantile(quantile)
print("Remove movies with vote_count lower than: " + str(min_vote_count))

### Number of movies to remove

In [ ]:
cond_to_remove = (
    (movies_df["genres"].astype(str) == "[]")
    | (movies_df["production_companies"].astype(str) == "[]")
    | (movies_df["budget"] == 0)
    | (movies_df["popularity"] == 0)
    | (movies_df["revenue"] == 0)
    | (movies_df["runtime"] == 0)
    | (movies_df["vote_average"] == 0)
    | (movies_df["vote_count"] < min_vote_count)
)
to_remove_movies_df = movies_df[cond_to_remove][
    [
        "title",
        "genres",
        "production_companies",
        "budget",
        "popularity",
        "revenue",
        "runtime",
        "vote_average",
        "vote_count",
    ]
]
print(("Number of movies to remove: " + str(len(to_remove_movies_df))))
to_remove_movies_df.head()

### Categorical features exploration

In [ ]:
values, _ = get_json_values_from_col(movies_df["genres"], "name")
values

In [ ]:
values, _ = get_json_values_from_col(movies_df["production_companies"], "name")
values[:20]

In [ ]:
values, _ = get_json_values_from_col(movies_df["keywords"], "name")
values[:20]

## Explore credits data

In [ ]:
list(credits_df.columns)

In [ ]:
credits_df.dtypes

### Explore json fields

#### Explore json keys

In [ ]:
col = credits_df["cast"]
values, _ = get_json_keys_from_col(col)
values[:20]

In [ ]:
col = credits_df["crew"]
values, _ = get_json_keys_from_col(col)
values[:20]

In [ ]:
values, _ = get_json_values_from_col(credits_df["crew"], "job")
values[:20]

#### Most relevant jobs of the crew
Manually selected from the whole list of jobs
- Creator
- Director
- Producer
- Co-Director
- Editor
- Executive Producer
- Co-Executive Producer
- Co-Producer
- Script Editor
- Executive In Charge Of Production
- Author
- Dialogue Editor
- Script
- Original Story

In [ ]:
values, _ = get_json_values_from_col(credits_df["crew"], "gender")
values